# t-SNE

t-SNE es una nueva técnica ganadora para la reducción de dimensiones y (sobretodo) la visualización de datos. t-SNE no solo captura la estructura local de la dimensión superior, sino que también preserva las estructuras globales de los datos como clústeres. Tiene una capacidad asombrosa para producir agrupaciones segregadas bien definidas. t-SNE se basa en la incrustación de vecinos estocásticos (SNE: Stochastic Neighbor Embedding). t-SNE se desarrolló para abordar algunos de los problemas en SNE.

## SNE
Para comprender t-SNE, necesitamos conocer SNE y sus deficiencias. SNE utiliza un enfoque probablístico para incrustar un conjunto de datos de alta dimensión en una dimensión inferior al preservar la estructura de la vecindad del conjunto de datos. Una distribución de probabilidad gaussiana centrada en cada punto es definida sobre todos los vecinos potenciales de este punto. SNE tiene como objetivo minimizar la diferencia en la distribución de probabilidad en una dimensión superior y una dimensión inferior.

Para cada objeto, $i$ y su vecino $j$, calculamos un $P_{i|j}$ que refleja la probabilidad de que $j$ sea vecino de $i$

$$P_{i|j} = \frac{\exp(-d_{ij}^2)}{\sum_{k\ne i}\exp(-d_{ik}^2)})$$

donde $d_{ik}^2$ es la diferencia entre el elemento $i$ y $k$ dado como entrada o calculado a partir del conjunto de datos proporcionado.

La diferencia entre $x_i$ y $x_j$ se puede calcular usando la siguiente fórmula

$$ d_{ij}^2 = \frac{||x_i-x_j||^2}{2\sigma_i^2},$$
donde $\sigma_i$ generalmente se calcula mediante una búsqueda binaria igualando la entropía de la distribución centrada en $x_i$ con la **perplejidad**, que se elige a mano. Este método genera una matriz de probabilidad asimétrica.

A continuación, SNE hace el mismo cálculo para cada $Y_i$ y $Y_j$ en la dimensión inferior con distribución de probabilidad gaussiana con $\sigma = 0.5$:

$$ q_{ij} = \frac{\exp(-||y_i-y_j||^2)}{\sum_k \exp(-||y_k-y_i||^2)}$$

Ahora, SNE intenta minimizar la diferencia entre estas dos distribuciones. Podemos calcular la diferencia entre dos distribuciones usando la [divergencia de Kullback-Liebler](https://www.countbayesie.com/blog/2017/5/9/kullback-leibler-divergence-explained). Para dos distribuciones discretas, $P$ y $Q$, la divergencia de KL viene dada por:

$$ D_{KL}(P||Q) = \sum_i P_{i}\log\frac{P_{i}}{Q_{i}}.$$

SNE define una función de costo basada en la diferencia entre $p_{ij}$ y $q_{ij}$ que viene dada por:

$$
C=\sum_{i} D_{KL}\left(P_{i} \| Q_{i}\right)=\sum_{i} \sum_{j} p_{j i} \log \frac{p_{j i}}{q_{j i}}
$$

Debido a que la divergencia de Kullback-Leibler no es simétrica, los diferentes tipos de error en las distancias por pares en el mapeo de baja dimensión no se ponderan por igual. En particular, existe un gran costo por usar puntos mapeados muy separados para representar puntos de datos cercanos (es decir, por usar un $q_{j i}$ pequeño para modelar un $p_{j i}$ grande), pero solo hay un costo pequeño por usar puntos mapeados cercanos para representar puntos de datos muy separados. Este pequeño costo proviene de desperdiciar parte de la masa de probabilidad en las distribuciones $Q$ relevantes. En otras palabras, **la función de costo SNE se enfoca en retener la estructura local de los datos en el mapeo** (para valores razonables de la varianza de Gauss en el espacio de alta dimensión, $\sigma_{i}$).

El gradiente para optimizar la función de costo viene dado por:
$$ \frac{\delta C}{\delta Y} = 2\Sigma_i (p_{ij}-q_{ij}+p_{ji}-q_{ji})(y_i - y_j)$$

Algunas de las limitaciones del enfoque de SNE son la matriz de probabilidad asimétrica $P$ y el problema del hacinamiento. Como se señaló anteriormente, la matriz de probabilidad $P$ es asimétrica. Suponga que un punto $x_i$ está lejos de otros puntos, su $p_{ij}$ será muy pequeño para todos los $j$. Por lo tanto, tendrá poco efecto en la función de costo y será difícil incrustarlo correctamente en la dimensión inferior.

Cualquier espacio euclidiano n-dimensional puede tener un objeto con a lo más $n + 1$ o vértices menos equidistantes. Ahora, cuando la dimensión intrínseca de un conjunto de datos es alta, digamos 20, y estamos reduciendo sus dimensiones de 100 a 2 o 3, nuestra solución se verá afectada por el problema del hacinamiento. La cantidad de espacio disponible para mapear puntos cercanos en 10 o 15 dimensiones siempre será mayor que el espacio disponible en 2 o 3 dimensiones. Para mapear correctamente los puntos cercanos, los puntos moderadamente distantes serán empujados demasiado lejos. Esto se comerá los huecos en los grupos originales y se verá como un solo grupo gigante.

Necesitamos repasar algunos temas más antes de pasar a t-SNE.

**Distribución t de Student**: la distribución t de Student es una función de distribución de probabilidad simétrica continua con colas pesadas. Tiene un solo parámetro *el grado de libertad*. A medida que aumenta el *grado de libertad*, se aproxima a la función de distribución normal. Cuando el *grado de libertad* = 1, toma la forma de función de distribución cauchy y su función de densidad de probabilidad viene dada por

$$ f(t) = \frac{1}{\pi (1+t^2)}$$

**Entropía**: la entropía es una medida de la información promedio contenida en un dato. Para una variable $x$ con $pdf$ $p(x)$, viene dada por

$$ H(x) = - \Sigma_{i}\,(\,p\,(x_i) \times  log_2(\,p(\,x_i\,)))$$

**Perplejidad**: en la teoría de la información, la perplejidad mide qué tan bien una distribución de probabilidad predice una muestra. Una perplejidad baja indica que la función de distribución es buena para predecir la muestra. La perplejidad está dada por

$$Perpx(x) = 2^{H(x)},$$ donde $H(x)$ es la entropía de la distribución.

## t-SNE algorithm

t-SNE se diferencia de SNE de dos maneras, primero utiliza una distribución t-student para medir la similitud entre los puntos $y_i$ y $y_j$ en la dimensión inferior, en segundo lugar, para la dimensión superior utiliza una distribución de probabilidad simétrica tal que $p_{ij} = p_{ji}$. Analicemos primero la distribución de probabilidad simétrica.

t-SNE define la probabilidad $P_{ij}$ como:
$$ p_{ij} = \frac{p_{ij}+p_{ij}}{2n}$$

Esta formulación asegura que $\sum_j p_{ij} \gt \frac{1}{2n}$ para cada $x_i$ y que $x_i$ haga una contribución significativa a la función de costo. El gradiente definido anteriormente se vuelve mucho más simple ahora:

$$\frac{\delta C}{\delta y_i} = 4 \sum_{ij}(p_{ij}-q_{ij})(y_i-y_j)$$
Con la distribución t de student que tiene un grado uno de libertad, la probabilidad por pares se puede definir como:
$$ q_{ij} = \frac{(1+||y_i-y_j||^2)^{-1}}{\sum_{k\ne i}(1+||y_i-y_k||^2)^{-1}}$$

Ahora el gradiente cambia a:

$$\frac{\delta C}{\delta y_i} = 4 \sum_{ij}(p_{ij}-q_{ij})(y_i-y_j)(1+||y_i-y_j||^2)^{-1}$$

### Pseudo-código para t-SNE

1. Se calcula la similitud por pares $p_{ij}$ para cada $i$ y $j$.
2. Se actualiza la probabilidad con $p_{ij}$ = $\frac{p_{ij}+p_{ji}}{2n}$
3. Se escoge una solución aleatoria $Y_0 = (y_0,y_1,y_2,...y_n)$ donde $y_i \in R^d$
4. Mientras no haya terminado:
   - calcular los valores similares por pares para $Y_0$
   - calcular el gradiente $\frac {\delta C}{\delta y_i}$
   - actualizar la solución $y_i^t = y_i^{t-1} + \eta \frac{\delta C}{\delta y_i} + (y_i^{t-1} - y_i^{t-2})$
   - if $t \gt \text{max_iter}$:
     - break
   - else
     - $t = t + 1$

Implementemos el algoritmo paso a paso.

#### Calcular las similitudes por pares
Para calcular similitudes por pares, necesitamos conocer la varianza $\sigma_i$ para el gaussiano centrado en $x_i$.

Uno podría pensar por qué no establecer un valor único de $\sigma_i$ para cada $x_i$. Es probable que la densidad de los datos varíe, necesitamos un $\sigma_i$ más pequeño para lugares con densidades más altas y un $\sigma_i$ más grande para lugares donde los puntos están lejos. La entropía de la distribución gaussiana centrada en $x_i$ aumenta a medida que aumenta $\sigma_i$. Para obtener el $\sigma_i$ necesitamos realizar una búsqueda binaria tal que la perplejidad de la distribución gaussiana centrada en $x_i$ sea igual a la perplejidad especificada por el usuario.

Ahora, si pensamos en cómo encaja la perplejidad en todo esto. Podemos pensar en la perplejidad como la medida uniforme del número de vecinos.

Entonces, calculamos la similitud por pares como:

$$p_{i|j} = \frac{\exp(-d_{ij}^2)}{\Sigma_{k\ne i}\exp(-d_{ij}^2)})$$

Una vez que se hayan calculado todas las similitudes por pares, actualizamos la similitud usando la siguiente regla:

- $p_{ii} = 0 $

- $p_{ij} = \frac{p_{ij}+p_{ji}}{2n}$

### Cálculo de incrustaciones de baja dimensión (low dimensional embedding)

Echemos un vistazo de cerca al gradiente obtenido.

$$\frac{\delta C}{\delta y_i} = 4 \sum_{ij}(p_{ij}-q_{ij})(y_i-y_j)(1+||y_i-y_j||^2)^{-1}$$

El gradiente t-SNE es mejor de dos maneras, primero, induce un alto costo cuando los puntos disímiles se modelan por pequeñas distancias de pares en una dimensión más baja y, en segundo lugar, aunque el costo inducido en el primer caso es alto, no se acerca al infinito debido a lo cual los grupos separados no se alejan mucho unos de otros.

La ecuación de actualización para $y_i$ se da como:

$$ y_i^t = y_i^{t-1} + \eta \frac{\delta C}{\delta y_i} + (y_i^{t-1} - y_i^{t-2})$$

donde $t$ es la iteración.

## TSNE con scikit-learn

La implementación ya se encuentra realizada de forma óptima en sklearn. Aquí la usaremos, pero pueden ver mejor la documentación para más detalles

¿Cómo se selecciona  la perplejidad para t-SNE?
La perplejidad es un parámetro importante del algoritmo t-SNE. Entenderemos sus efectos usando un conjunto de datos simple (iris):

In [41]:
from sklearn.manifold import TSNE

In [43]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from sklearn import datasets

iris = datasets.load_iris().data
target = datasets.load_iris().target

def plotter(x):
    
    Y = TSNE(n_components=2, perplexity=x).fit_transform(iris)
    
    setosa = Y[target==0]
    versicolor = Y[target==1]
    virginica = Y[target==2]
    plt.scatter(setosa[:,0], setosa[:,1], c="b",label="setosa")
    plt.scatter(versicolor[:,0], versicolor[:,1], c="g",label="versicolor")
    plt.scatter(virginica[:,0], virginica[:,1], c="r",label="verginica")
    plt.legend()
    plt.show()

interact(plotter, x= widgets.IntSlider(min=5, max=40, value=10, step=5))

interactive(children=(IntSlider(value=10, description='x', max=40, min=5, step=5), Output()), _dom_classes=('w…

<function __main__.plotter(x)>

Si cambiamos el valor de perplejidad del control deslizante, verá que los clusters cambian drásticamente. Para que t-SNE sea significativo, tenemos que elegir el valor correcto de perplejidad. La perplejidad equilibra los aspectos locales y globales del conjunto de datos. Un valor muy alto conducirá a la fusión de grupos en un solo grupo grande y un valor bajo producirá muchos grupos pequeños cercanos que no tendrán sentido.

### Ventajas e inconvenientes

t-SNE funciona bien para conjuntos de datos no lineales. Funciona mucho mejor que otros algoritmos no lineales. Los problemas surgen cuando las dimensiones intrínsecas son más altas, es decir, más de 2-3 dimensiones. t-SNE tiende a quedarse atascado en óptimos locales.

# Referencias

[1] L.J.P. van der Maaten and G.E. Hinton. Visualizing High-Dimensional Data Using t-SNE. Journal of Machine Learning Research 9(Nov):2579-2605, 2008. [PDF](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf)

[2] http://distill.pub/2016/misread-tsne

# Ejercicio

Cargue el dataset de digits de sklearn y realice una visualización con t-SNE visualizando los diferentes clústers.